# Simulator Integration

In this tutorial we show how to integrate a simulator into OpenSBT. We provide here two examples: 

- First we integrate the simulator for the Dummy Case study to get familiar with the integration steps. 
- Then we integrate the [CARLA](carla.org) simulator for a more practical example.

OpenSBT offers the abstract class [`Simulator`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/simulation/simulator.py) which needs to be implemented to use a concrete simulator. I.e. we need to implement the [simulate]() method of the abstract class. Simulate receives as input a list of scenario instances, and outputs for each scenario instance a `SimulationOutput`. The implementation of `simulate` is simulator specific. Following steps need to be implemented:

1. Passing the scenario instance(s) to the simulator and executing the SUT on the scenario(s).
2. Converting the simulation output returned by the simulator to the format understandable by OpenSBT.

Consider the structure of the generic `SimulationOutput` in OpenSBT is defined as follow:
```json 
    {
        "simTime" : 3,
        "times": [1.0,2.0,3.0],
        "location": { 
                "ego" : [(1,1),(2,2),(3,3)],
                "adversary" : [(4,1),(4,2),(4,3)
         },
        "velocity": {                            //velocity for each dimension x,y,z
                "ego" : [(0.5,0.5,0.5),(0.5,0.5,0.5),(0.5,0.5,0.5)],
                "adversary" : [(0.9,0.9,0.9),(0.9,0.9,0.9),(0.9,0.9,0.9)],
         },
        "speed" : {                              //norm of the velocity vectors
            "ego" : [1,3,3],
            "adversary" : [3,2,5],

         },
        "acceleration": {
                "ego": [],
                "adversary": []
         },
        "yaw": {
            "ego": [],
            "adversary": []
         },
        "collisions": [],       // dictionary: timestep of collision -> actor involved
        "actors" : {              // map: type to name (for visualization)
            "ego": "ego",
            "adversary": "adversary",        // there is a main adversary agent selected
            "vehicles" : [],
            "pedestrians" : []
        },
        "otherParams" : {   //simulator specific outputs
            "isCollision" : True,
            "height" : "2",
            "width" : "5
        }
    }
```

## Dummy Simulator Integration

For the Dummy Eample (s. [02_Experiment_Definition.ipynb]()) the simulator simulates ego and the pedestrian following a linear trajectoy defined by the *initial position* and a *orientation*. A small perturbation to the trajectory is added so simulate uncertainty.

Further ego has an AEB integrated that scans for objects within a predefined range. When an actor is detected, the ego vehicle will stop and continue moving when no objects are detected anymore within the range.

We see below an implementation for the integration of this simulator.

 The `DummySimulator` class sublasses the `Simulator` class by implementing the `simulate` method (line 21). Simulate receives a list of test cases in the format a list of arrays. Every test case is represented by an n-dimensional array of floating numbers (here: individual), in our case the search space is 4-dimensonal, since a test case is represented by **ego_orientation**, **ego_velocity**, **ped_orientation** and **ped_velocity**. The corresponding variable names are passed in line 31.
 
Each test input is further processed in line 57/58ff. In this example we interpolate the trajectories for ego and pedestrian. In contrast, in a _real_ simulator one would use the `scenario_path` which holds an abstract scenario to instantiate a _concrete_ scenario. Then one would pass it to the simulator and trigger the execution of the SUT._(See following example)_

In our example, we finally only need to create a JSON object that adheres to the format of the `SimulationOutput`class, as done in line 118ff. Then we only need to return a `SimulationOutput` instance which is constructed from the given JSON file (line 175).

```python
from typing import List
from simulation.simulator import Simulator, SimulationOutput
from math import sin, cos, pi, ceil
import numpy as np
from random import random
from utils.geometric import *
import logging as log
from utils import geometric
import json

''' Simulation based on the linear interpolation of the motions of two actors. 
    Ego contains an AEB which scans for nearby vehicles below some distance threshold.
'''
class DummySimulator(Simulator):
    DETECTION_THRESH = 2     # threshold in meters where other actors can be detected
    RANDOMNESS_BIAS = 0.1    # noise to be added to positions
    
    ''' Simulates a set of scenarios of the form (ego_orientation, ego_velocity, ped_orientation, ped_velocity) and returns the output''' 
    @staticmethod
    def simulate(list_individuals, 
                 variable_names, 
                 scenario_path: str, 
                 sim_time: float, 
                 time_step: float = 1,
                 do_visualize: bool = False
        ) -> List[SimulationOutput]:
        results = []
        for ind in list_individuals:
            simout = DummySimulator.simulate_single(ind, 
                                                    variable_names, 
                                                    filepath=scenario_path, 
                                                    sim_time=sim_time,
                                                    time_step=time_step)
            results.append(simout)
        return results

    @staticmethod
    def simulate_single(vars, 
                        variable_names, 
                        filepath, 
                        sim_time: float, 
                        time_step: float, 
                        detection_dist = DETECTION_THRESH, 
                        randomness_bias = RANDOMNESS_BIAS
    ) -> SimulationOutput:

        egoInitialVelocity = vars[1]
        pedInitialVelocity = vars[3]
        egoOrientation = vars[0]
        pedOrientation = vars[2]
        start_pos_ped = [2, 10] # Fixed starting position
        start_pos_ego = [0, 0] # Fixed starting position

        # first plan the motion of ego, get for each time step predicted position/velocity

        plan_ego_traj= planMotion(start_pos_ego, egoOrientation, egoInitialVelocity, sim_time, time_step)
        plan_adv_traj = planMotion(start_pos_ped, pedOrientation, pedInitialVelocity, sim_time, time_step)

        line_ego_points = (plan_ego_traj[1:3, 0], plan_ego_traj[1:3, 1])
        line_ped_points = (plan_adv_traj[1:3, 0], plan_adv_traj[1:3, 1])

        colpoint = geometric.intersection(list(line_ego_points), list(line_ped_points))

        n_steps = plan_ego_traj.shape[1]
        
        # iterate through each time step checking other actors nearby to avoid collision
        real_ego_traj = np.asarray([
                                    [plan_ego_traj[0,0]],
                                    [plan_ego_traj[1,0]],
                                    [plan_ego_traj[2,0]],
                                    [plan_ego_traj[3,0]],
                                    [plan_ego_traj[4,0]]
        ])

        # first position is real
        k = 1
        for i in range(1,n_steps):
            #log.info(f"current index: {i}")
            current_size = real_ego_traj[1,:]
            if len(current_size) < n_steps:
                pos_ego = [real_ego_traj[1,-1],real_ego_traj[2,-1]]
                pos_others = [[plan_adv_traj[1,i],plan_adv_traj[2,i]]]
                if are_actors_nearby(pos_ego, pos_others, detection_dist=detection_dist):
                    # plan make a step, ego vehicle trajectory in the form (Time, X, Y, V, Yaw)
                    # stop for some time steps
                    # t_stop = 3
                    # for t in range(0, n_steps - current_size - t_stop)):
                    new_t = real_ego_traj[0, i - 1] 
                    new_x =  real_ego_traj[1, i - 1] 
                    new_y =  real_ego_traj[2, i - 1] 
                    new_v = 0 # break
                    new_yaw = real_ego_traj[4, i - 1] 
                    new_state =  np.asarray([[new_t],[new_x],[new_y], [new_v], [new_yaw]])
                    real_ego_traj = np.append(real_ego_traj, new_state, axis = 1)
                    # waiting_steps += t_stop - 1
                    # log.info(f"waiting_steps: {waiting_steps}")
                else:     
                    new_t =  plan_ego_traj[0, k] 
                    new_x =  plan_ego_traj[1, k] 
                    new_y =  plan_ego_traj[2, k] 
                    new_v = plan_ego_traj[3, k]
                    if real_ego_traj[3, k-1] == 0:
                        new_v = plan_ego_traj[3,k]/2
   
                    new_yaw = plan_ego_traj[4,k]     
                    new_state =  np.asarray([[new_t],[new_x],[new_y], [new_v], [new_yaw]])
                    real_ego_traj = np.append(real_ego_traj, new_state, axis = 1)
                    k += 1
                #log.info(f"len(real_ego_traj[3, :]): {len(real_ego_traj[3, :])}")

        # add some noise to location vector
        ego_location = [pos for pos in zip(list(real_ego_traj[1, :]), list(real_ego_traj[2, :]))]
        obj_location = [pos for pos in zip(list(plan_adv_traj[1, :]), list(plan_adv_traj[2, :]))]

        ego_location = [(pos[0] +random()*randomness_bias,pos[1] +random()*randomness_bias) for pos in ego_location]

        result = {
            "simTime": 0,
            "times": list(real_ego_traj[0, :]),
            "location": {"ego": ego_location,
                         "adversary": obj_location},

            "velocity": {"ego": list(real_ego_traj[3, :]),
                         "adversary": list(plan_adv_traj[3, :]),
                         },

            "speed": {"ego": list(real_ego_traj[3, :]),
                         "adversary": list(plan_adv_traj[3, :]),
                         },

            "acceleration": {"ego": list(real_ego_traj[3, :]),
                         "adversary": list(plan_adv_traj[3, :]),
                         },

            "yaw": {"ego": list(real_ego_traj[4, :]),
                           "adversary": list(plan_adv_traj[4, :]),
                           },

            "collisions": [],
            "actors" : { 
                "ego": "ego",
                "adversary": "adversary",
                "vehicles" : [],
                "pedestrians" : []
            },
            "otherParams": {}
        }

        otherParams = {}

        # Collision calculation (Only for fixed trajectories)
        if colpoint != []:
            dist_ego_colpoint = geometric.dist(colpoint, list(real_ego_traj[1:3, 0]))
            dist_ped_colpoint = geometric.dist(colpoint, list(plan_adv_traj[1:3, 0]))

            t_col_ego = dist_ego_colpoint / egoInitialVelocity
            t_col_ped = dist_ped_colpoint / pedInitialVelocity

            # collision occurs when both objects reach line crossing at
            # the same time (with some tolerance)

            t_tolerance = 1;  # time tolerance for missed collision
            if (t_col_ego - t_col_ped) < t_tolerance:
                otherParams['isCollision'] = True
            else:
                otherParams['isCollision'] = False
            result["collisions"] = [[2]]
            result["otherParams"] = otherParams

        else:
            otherParams['isCollision'] = False
        result["otherParams"] = otherParams
      
        return SimulationOutput.from_json(json.dumps(result))
    
    ''' Checks if other actors are within a predefined distance.'''    
    def _are_actors_nearby(pos_ego, pos_others, detection_dist=3):
        for i in range(0, len(pos_others)):
            if geometric.dist(pos_ego,pos_others[i]) < detection_dist:
                return True
        return False   

    '''
        Linear motion planner

        Output: 
            Ego vehicle trajectory in the form (Time, X, Y, V, Yaw)
    '''
    def _planMotion(startingPosition, orientation, velocity, simTime, samplingTime):
        theta = orientation
        T = simTime
        v = velocity
        t = samplingTime

        S = v * T
        distX = cos(theta * pi / 180) * S
        distY = sin(theta * pi / 180) * S
        nSteps = ceil(T / t)

        asize = nSteps + 1
        arrayTime = np.linspace(0, T, asize)
        arrayX = np.linspace(startingPosition[0], startingPosition[0] + distX, asize)
        arrayY = np.linspace(startingPosition[1], startingPosition[1] + distY, asize)
        arrayV = v * np.ones(asize, dtype=np.int64)
        arrayYaw = theta * np.ones(arrayTime.size)
        return np.concatenate((arrayTime, arrayX, arrayY, arrayV, arrayYaw)).reshape(5, asize)
    ```

## CARLA Integration

In this example we explain how to integrate the CARLA Simulator. As in the example before, a specific ```CarlaSimulation``` class is created. In line 30 a concrete scenario instance is created based on the scenario template provided via ```scenario_path``` and the generated test inputs hold in ```list_individuals```. Here a test input has not a fixed length, the user can configure which and how many parameters constitute the search space.
We trigger the in line 35 the execution of the CARLA and the SUT on this test input. We outsourced the code for this process in a dedicate module called [CarlaRunner](https://git.fortiss.org/fortissimo/ff1_testing/ff1_carla) for better maintenance. Note, that concrete scenarios are stored in the SCENARIO_DIR folder and not directly passed. To inspect how this execution is triggered please check the [balancer](https://git.fortiss.org/opensbt/carla-runner/-/blob/master/src/carla_simulation/balancer.py?ref_type=heads) class of the CARLA Runner repository. 

When the simulation in CARLA has finished, the balancer outputs the simulation results via a JSON file implementing the ```SimulationOutput``` format shown before (line 43).

```python
from pathlib import Path
from typing import List, Dict
from simulation.simulator import Simulator, SimulationOutput
import logging as log
import json
import os
from model_ga.individual import Individual
from carla_simulation.balancer import Balancer

SCENARIO_DIR = "/tmp/scenarios"

class CarlaSimulator(Simulator):

    _balancer = None

    ''' Simulates a set of scenarios and returns the simulation traces'''
    @staticmethod
    def simulate(
        list_individuals: List[Individual],
        variable_names: List[str],
        scenario_path: str,
        sim_time: float,
        time_step: float,
        do_visualize: bool = False
    ) -> List[SimulationOutput]:
        xosc = scenario_path
        try:
            for ind in list_individuals:
                instance_values = [v for v in zip(variable_names,ind)]
                CarlaSimulator.create_scenario_instance_xosc(
                            xosc, 
                            dict(instance_values), 
                            outfolder=SCENARIO_DIR)
            if CarlaSimulator._balancer is None:
                CarlaSimulator._balancer = Balancer(
                    directory = SCENARIO_DIR,
                    jobs = 1,
                    visualization = do_visualize,
                    agent = 'NPCAgent'
                )
                CarlaSimulator._balancer.start()
            # Execute SUT on scenario instances
            outs = CarlaSimulator._balancer.run()
            results = []
            for out in outs:
                simout = SimulationOutput.from_json(json.dumps(out))
                results.append(simout)
        except Exception as e:
            raise e
        finally:
            # Remove all created .xosc scenario
            file_list = [ f for f in os.listdir(SCENARIO_DIR) if f.endswith(".xosc") ]
            for f in file_list:
                os.remove(os.path.join(SCENARIO_DIR, f))
        return results

    ''' Replaces parameter values in parameter declaration section in generic .xosc file 
    by provided values '''
    @staticmethod
    def _create_scenario_instance_xosc(filename: str, values_dict: Dict, outfolder=None):
        import xml.etree.ElementTree as ET
        xml_tree = ET.parse(filename)

        parameters = xml_tree.find('ParameterDeclarations')
        for name, value in values_dict.items():
            for parameter in parameters:
                if parameter.attrib.get("name") == name:
                    parameter.attrib["value"] = str(value)

        # # Write the file out again
        if outfolder is not None:
            Path(outfolder).mkdir(parents=True, exist_ok=True)
            filename = outfolder + os.sep + os.path.split(filename)[1]
        splitFilename =  os.path.splitext(filename)
        newPathPrefix = splitFilename[0]
        ending = splitFilename[1]

        suffix = ""
        for k,v in values_dict.items():
            suffix = suffix + "_" + str(v)

        newFileName  = newPathPrefix + suffix + ending
        xml_tree.write(newFileName)
        return newFileName
```